## create_datasets를 먼저 돌리고 실행

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pandas_datareader as pdr 

# Fama - French 5요인
- 참고 논문: http://kiss.kstudy.com/thesis/thesis-view.asp?key=3450399

## Size 요소 가져오기

In [2]:
idx = pd.IndexSlice

In [3]:
stock_data = pd.read_hdf('assets.h5', 'finance_datareader/prices')
market_cap_rank_data = stock_data[['marcap', 'rank']]

In [4]:
total_asset = pd.read_csv('IFRS/TotalAssets.csv', encoding='CP949')

In [5]:
# 종목코드를 6자리로 맞춰줌 -> 파이썬 고질병 '000660'을 숫자로 자동 인식하면 앞의 000을 날려버림 이거를 string으로 변경해서 복구
ticker = total_asset['거래소코드'].apply("{0:0>6}".format)

In [6]:
# 금융업 제외한 재무제표랑 market cap 데이터랑 겹치는 종목 추출
intersect_ticker = market_cap_rank_data.unstack('date').index.intersection(ticker)

In [7]:
size_factor = (market_cap_rank_data
                .unstack('date')
                .loc[intersect_ticker,:].stack('date')
                .reorder_levels(['date','ticker'])
                .sort_index())

In [8]:
size_factor = size_factor.astype('int64')

In [9]:
# 월별로 시가총액 순위를 구하고 6월달 시가총액 순위만 출력
month_rank = size_factor['marcap'].unstack('ticker').resample('M').last().rank(axis=1)
six_month_rank = month_rank[month_rank.index.month == 6].stack('ticker').to_frame()
six_month_rank = six_month_rank.rename(columns={0:'rank'})
six_month_rank = six_month_rank.astype('int32')

In [10]:
# Monthly Period로 변경
six_month_rank.index = six_month_rank.index.set_levels(six_month_rank.index.levels[0].to_period('M'), level=0)

In [11]:
# 일자별 시가총액 순위 데이터
size_factor.head(5)

marcap  rank
date       ticker                    
1995-05-02 000020   76796000000   157
           000050   84854000000   148
           000070  247896000000    50
           000100  145536000000    86
           000150   57455244000   189

In [12]:
six_month_rank.head(5)

rank
date    ticker      
1995-06 000020   142
        000050   143
        000070   180
        000100   169
        000150   132

## B/M Book to Market Ratio 구하기
- 시가총액은 연도 말을 기준으로 한다.
- 재무제표는 사업보고서이고 공시 날짜는 6월 1일로 통일한다.

In [13]:
def get_ifrs_data(data_path: str):
    data = pd.read_csv(data_path, encoding='CP949')
    data.fillna(0, inplace=True)
    data['거래소코드'] = data['거래소코드'].apply("{0:0>6}".format)
    data['회계년도'] = pd.to_datetime(data['회계년도'])
    data = data.set_index(['회계년도','거래소코드']).sort_index()
    # 회계년도 안맞는 것들 resample을 이용해 연말로 회계년도 통일
    data = data.unstack('거래소코드').resample('Y').last().stack('거래소코드')
    data.index = data.index.set_levels(data.index.levels[0].to_period('M'), level=0)
    col_list = data.columns
    before_ifrs = data.loc[idx[:'2006',:],:][col_list[-2]]
    after_ifrs = data.loc[idx['2007':,:],:][col_list[-1]]
    data = pd.concat([before_ifrs, after_ifrs]).to_frame()
    data.columns = [col_list[-2]]
    data.index.names = ['date','ticker']
    data.sort_index(inplace=True)
    return data.copy()

In [14]:
idx = pd.IndexSlice

In [15]:
common_stock_capital = get_ifrs_data('IFRS/CommonStock.csv')

In [16]:
common_stock_capital

보통주자본금(천원)
date    ticker             
1981-12 000020    2808000.0
        000040    3450000.0
        000050    2400000.0
        000070    9500000.0
        000080    3000000.0
...                     ...
2020-12 363280   18750451.0
        375500          0.0
        378850          0.0
        900140          0.0
        950210  152779419.0

[20876 rows x 1 columns]

In [17]:
capital_surplus = get_ifrs_data('IFRS/CapitalSurplus.csv')

In [18]:
capital_surplus

자본잉여금(*)(천원)
date    ticker              
1981-12 000020        4410.0
        000040     1903860.0
        000050    12797820.0
        000070     9538010.0
        000080     8837667.0
...                      ...
2020-12 363280   528596133.0
        375500           0.0
        378850           0.0
        900140           0.0
        950210           0.0

[20876 rows x 1 columns]

In [ ]:
retained_earnings = get_ifrs_data('IFRS/RetainedEarnings.csv')

In [ ]:
retained_earnings

In [ ]:
deferred_tax_liabilities = get_ifrs_data('IFRS/DeferredTaxLiabilities.csv')

In [ ]:
market_cap_data = stock_data['marcap'].to_frame()

In [ ]:
market_cap_data = market_cap_data.unstack('ticker').resample('Y').last().stack('ticker')

In [ ]:
market_cap_data.index = market_cap_data.index.set_levels(market_cap_data.index.levels[0].to_period('M'), level=0)

In [ ]:
market_cap_data

In [ ]:
book_to_market_data = pd.concat([common_stock_capital, capital_surplus, retained_earnings, deferred_tax_liabilities, market_cap_data], axis=1)
book_to_market_data.sort_index(inplace=True)

In [ ]:
book_to_market_data

In [ ]:
# 시가총액이 있는 년도부터 시작
book_to_market_data = book_to_market_data.loc[idx['1995':'2021',:],:]

In [ ]:
# 결측치 제거
book_to_market_data.dropna(inplace=True)

### Book to Market Ratio 구하기

In [ ]:
# B/M 계산
columns_list = book_to_market_data.columns
book_value = book_to_market_data.loc[:,columns_list[:-1]].sum(axis=1)
book_to_market_ratio = book_value.div(book_to_market_data['marcap']).to_frame()
book_to_market_ratio.columns = ['BM']


### 회계년도랑 공시년도가 차이가 나므로 회계년도에서 6개월 뒤로 미룬다.
- 년말 12월 기준의 회계가 있어도 공시하기까지 시간이 걸린다. 이부분을 고려

In [ ]:
def offset_6_month(data: pd.DataFrame) -> pd.DataFrame:
    '''
        Input DataFrame E.g
        Multi Index DataFrame
                      price
        date  ticker
     1995-12  000660  1000
              005930  20000
              003229  3004000
              
        
        Output DataFrame E.g
                         price
        date  ticker
     1996-06  000660  1000
              005930  20000
              003229  3004000
        
    '''
    date_index = data.index.levels[0].to_timestamp() + pd.DateOffset(months=6)
    data.index = data.index.set_levels(date_index.to_period('M'), level=0)
    return data 

## B/M Ratio 공시날짜를 위해서 6개월 미루기

In [ ]:
book_to_market_ratio = offset_6_month(book_to_market_ratio)

In [ ]:
book_to_market_ratio

## 수익성 지표 계산
- OP = 영업이익 / 자기자본 장부가치
    + 영업이익 = 매출액 - 매출원가 - 이자비용 - 판관비

#### 매출액

In [ ]:
net_sales = get_ifrs_data('IFRS/NetSales.csv')

In [ ]:
net_sales

#### 매출원가

In [ ]:
cost_of_sales = get_ifrs_data('IFRS/CostOfSales.csv')

In [ ]:
cost_of_sales

#### 이자비용

In [ ]:
interest_expense = get_ifrs_data('IFRS/InterestExpenses.csv')

In [ ]:
interest_expense

### 판매비와 관리비

In [ ]:
selling_and_admin_expense = get_ifrs_data('IFRS/Selling_and_administrative_expenses.csv')

In [ ]:
selling_and_admin_expense

In [ ]:
op_data = pd.concat([net_sales, cost_of_sales, interest_expense, selling_and_admin_expense], axis=1)

In [ ]:
op_data.head(5)

In [ ]:
col_list = op_data.columns

In [ ]:
# 위의 식대로 계산
op = (op_data[col_list[0]] - op_data[col_list[1:]].sum(axis=1)).div(book_value).to_frame()

In [ ]:
op.columns = ['OP']
op.dropna(inplace=True)

In [ ]:
op = offset_6_month(op)

In [ ]:
op 

## 자본투자
- t-1년 12월 말의 총자산에서 t-2년 12월 말의 총 자산을 차감한 총자산증가액
    + inv = TotalAsset(t-1) / TotalAsset(t-2) - 1

### 총 자산

In [ ]:
total_asset = get_ifrs_data('IFRS/TotalAssets.csv')

In [ ]:
total_assett_unstack = total_asset.unstack('ticker')

In [ ]:
inv = (total_assett_unstack / total_assett_unstack.shift(1)).stack('ticker') - 1

In [ ]:
inv = offset_6_month(inv)

In [ ]:
inv.columns = ["INV"]

In [ ]:
inv

## Size-B/M, Size-OP, Size-Inv Portfolio 구성
- 가격 데이터가 1995년부터 있으므로 1995년부터 Portfolio 구성
- Value weight portfolio

기업규모
- 하위 50%, 상위 50%


B/M
- 상위 33%, 중위 33%, 하위 33%


수익성
- 상위 33%, 중위 33%, 하위 33%


자본투자
- 하위 33%, 중위 33%, 상위 33%

#### Size - B/M 포트폴리오
|Size/ BM|Small|Big|
|---|---|---|
|High BM|SH|BH|
|2|SN|BN|
|Low BM|SL|BL|

#### Size - OP 포트폴리오
|Size/ OP|Small|Big|
|---|---|---|
|High OP|SR|BR|
|2|SN|BN|
|Low OP|SW|BW|

#### Size - Inv 포트폴리오
|Size/ Inv|Small|Big|
|---|---|---|
|High Inv|SC|BC|
|2|SN|BN|
|Low Inv|SA|BA|

### Equal Weight Portfolio로 구성 

#### SMB 구하기
- 2분위수로 Size Factor로 나누고 이에 따라서

In [ ]:
# 시간 인덱스 가져오기
period_index = book_to_market_ratio.unstack('ticker').index

### 수정 가격 데이터 가져오기

In [ ]:
price_data = pd.read_hdf('assets.h5','finance_datareader/prices')[['close', 'adj_close', 'adj_volume', 'marcap']]

In [ ]:
return_data = (price_data[['marcap']]
                                .unstack('ticker')
                                .fillna(method='bfill')
                                .fillna(0)
                                .resample('M')
                                .last()
                                .pct_change()
                                .stack('ticker')
                                
              )

In [ ]:
return_data.columns = ['return']

In [ ]:
return_data.index = return_data.index.set_levels(return_data.index.levels[0].to_period('M'), level=0)

In [ ]:
return_data

### 1년 Size - B/M Portfolio 수익률 먼저 만들어 보기

In [ ]:
# 처음 월들을 기준으로 정하기
first_month = period_index[0]
print(first_month)

In [ ]:
rank = six_month_rank.loc[idx[first_month,:],:]['rank']

In [ ]:
rank

#### 2분위수로 Size Factor 나누기
- Size를 먼저

In [ ]:
rank_qauntile = pd.qcut(rank, q=2, labels=['small','big']).to_frame()

In [ ]:
rank_qauntile

In [ ]:
big_marcap_ticker = rank_qauntile[rank_qauntile['rank'].astype(str) == 'big'].index.get_level_values('ticker')

In [ ]:
small_marcap_ticker = rank_qauntile[rank_qauntile['rank'].astype(str) == 'small'].index.get_level_values('ticker')

### Portfolio 기간 가져오기

In [ ]:
def get_year_return(date:pd.Period, ticker_range):
    start = date.to_timestamp()
    end = start + pd.DateOffset(months=13) - pd.DateOffset(days=1)

    year_return = (return_data.loc[idx[start:end, ticker_range],:]
                                        .unstack('ticker')
                                        .mean(axis=1)
                                        .mul(100)
                    )
    return year_return

In [ ]:
big_marcap_year_return = get_year_return(first_month, big_marcap_ticker)

In [ ]:
big_marcap_year_return

In [ ]:
small_marcap_year_return = get_year_return(first_month, small_marcap_ticker)

In [ ]:
smb = small_marcap_year_return - big_marcap_year_return
print(smb)

### 이제 모든 기간의 SMB 수익을 구해본다.

In [ ]:
# 멀티 쓰레드 풀 이용
from concurrent import futures
import concurrent

# 이 작업은 CPU 리소스와 시간이 많이 들어갑니다.
smb_list = []
def calculate_return(period):
    print(period)
    rank = six_month_rank.loc[idx[period,:],:]['rank']
    rank_qauntile = pd.qcut(rank, q=2, labels=['small','big']).to_frame()
    
    big_marcap_ticker = rank_qauntile[rank_qauntile['rank'].astype(str) == 'big'].index.get_level_values('ticker')
    small_marcap_ticker = rank_qauntile[rank_qauntile['rank'].astype(str) == 'small'].index.get_level_values('ticker')
    
    big_marcap_year_return = get_year_return(period, big_marcap_ticker) 
    small_marcap_year_return = get_year_return(period, small_marcap_ticker) 
    smb = (small_marcap_year_return - big_marcap_year_return).to_frame()
    smb.columns = ['SMB']
    smb_list.append(smb)
    
with futures.ThreadPoolExecutor(10) as executor:
    executor.map(calculate_return, period_index)

In [ ]:
SMB = pd.concat(smb_list)

In [ ]:
SMB.sort_index(inplace=True)

### Kospi Index 데이터 가져오기

In [ ]:
kospi = pd.read_hdf('assets.h5', 'finance_datareader/kospi')
kospi.rename(columns={'close':'Mkt-RF'}, inplace=True)
kospi = kospi[kospi.index >= period_index[0].to_timestamp()]

In [ ]:
kospi = kospi['Mkt-RF'].resample('M').last().pct_change().mul(100).to_frame().dropna()
kospi.index = kospi.index.to_period('M')

### 코스피 Index랑 SMB 비교

In [ ]:
(SMB['SMB']/100 + 1).cumprod().plot()
(kospi['Mkt-RF']/100 + 1).cumprod().plot()
plt.legend()

## HML 월별 수익률 구하기

#### Size - B/M 포트폴리오
|Size/ BM|Small|Big|
|---|---|---|
|High BM|SH|BH|
|2|SN|BN|
|Low BM|SL|BL|



구하는 식
- HML = (SH + BH)/2 - (SL + BL)/2  

In [ ]:
period = period_index[0]

In [ ]:
big_bm_data = book_to_market_ratio.loc[idx[period, big_marcap_ticker],:]['BM']
small_bm_data = book_to_market_ratio.loc[idx[period, small_marcap_ticker],:]['BM']

### B/M에 따라 3분위수로 나눈다.

In [ ]:
big_bm_quantile = pd.qcut(big_bm_data, q=3, labels=['BL', 'BN', 'BH']).to_frame()

In [ ]:
small_bm_quantile = pd.qcut(small_bm_data, q=3, labels=['SL', 'SN', 'SH']).to_frame()

In [ ]:
bh_ticker = big_bm_quantile[big_bm_quantile['BM'].astype(str) == 'BH'].index.get_level_values('ticker')
sh_ticker = small_bm_quantile[small_bm_quantile['BM'].astype(str) == 'SH'].index.get_level_values('ticker')

In [ ]:
bl_ticker = big_bm_quantile[big_bm_quantile['BM'].astype(str) == 'BL'].index.get_level_values('ticker')
sl_ticker = small_bm_quantile[small_bm_quantile['BM'].astype(str) == 'SL'].index.get_level_values('ticker')

In [ ]:
hml = (get_year_return(period, bh_ticker) + get_year_return(period, sh_ticker))/2 - (get_year_return(period, bl_ticker) + get_year_return(period, sl_ticker))/2

### 전체 기간의 HML을 구해본다.

In [ ]:
hml_list = []
def calculate_return(period):
    print(period)
    big_bm_data = book_to_market_ratio.loc[idx[period, big_marcap_ticker],:]['BM']
    small_bm_data = book_to_market_ratio.loc[idx[period, small_marcap_ticker],:]['BM']
    
    # 3분위수로 나눈다.
    big_bm_quantile = pd.qcut(big_bm_data, q=3, labels=['BL', 'BN', 'BH']).to_frame()
    small_bm_quantile = pd.qcut(small_bm_data, q=3, labels=['SL', 'SN', 'SH']).to_frame()
    
    bh_ticker = big_bm_quantile[big_bm_quantile['BM'].astype(str) == 'BH'].index.get_level_values('ticker')
    sh_ticker = small_bm_quantile[small_bm_quantile['BM'].astype(str) == 'SH'].index.get_level_values('ticker')
    
    bl_ticker = big_bm_quantile[big_bm_quantile['BM'].astype(str) == 'BL'].index.get_level_values('ticker')
    sl_ticker = small_bm_quantile[small_bm_quantile['BM'].astype(str) == 'SL'].index.get_level_values('ticker')
    
    hml = (get_year_return(period, bh_ticker) + get_year_return(period, sh_ticker))/2 - (get_year_return(period, bl_ticker) + get_year_return(period, sl_ticker))/2
    hml = hml.to_frame()
    hml.columns = ['HML']
    hml_list.append(hml)
    
for period in period_index:
    hml_list.append(calculate_return(period))

In [ ]:
HML = pd.concat(hml_list).sort_index()

In [ ]:
(SMB['SMB']/100 + 1).cumprod().plot()
(HML['HML']/100 + 1).cumprod().plot()
(kospi['Mkt-RF']/100 + 1).cumprod().plot()
plt.legend()

## RMW 월별 수익률 구하기¶

#### Size - OP 포트폴리오
|Size/ OP|Small|Big|
|---|---|---|
|High OP|SR|BR|
|2|SN|BN|
|Low OP|SW|BW|


구하는 식
- RMW = (SR + BR)/2 - (SW + BW)/2  

### 구하는 식의 HML이랑 비슷하므로 바로 코드로 넘어간다.

In [ ]:
rmw_list = []

def calculate_return(period):
    print(period)
    big_op_data = op.loc[idx[period, big_marcap_ticker],:]['OP']
    small_op_data = op.loc[idx[period, small_marcap_ticker],:]['OP']
    
    # 3분위수로 나눈다.
    big_op_quantile = pd.qcut(big_op_data, q=3, labels=['BW', 'BN', 'BR']).to_frame()
    small_op_quantile = pd.qcut(small_op_data, q=3, labels=['SW', 'SN', 'SR']).to_frame()
    
    br_ticker = big_op_quantile[big_op_quantile['OP'].astype(str) == 'BR'].index.get_level_values('ticker')
    sr_ticker = small_op_quantile[small_op_quantile['OP'].astype(str) == 'SR'].index.get_level_values('ticker')
    
    bw_ticker = big_op_quantile[big_op_quantile['OP'].astype(str) == 'BW'].index.get_level_values('ticker')
    sw_ticker = small_op_quantile[small_op_quantile['OP'].astype(str) == 'SW'].index.get_level_values('ticker')
    
    rmw = (get_year_return(period, br_ticker) + get_year_return(period, sr_ticker))/2 - (get_year_return(period, bw_ticker) + get_year_return(period, sw_ticker))/2
    rmw = rmw.to_frame()
    rmw.columns = ['RMW']
    return rmw 

for period in period_index:
    rmw_list.append(calculate_return(period))

In [ ]:
RMW = pd.concat(rmw_list).sort_index()

In [ ]:
(SMB['SMB']/100 + 1).cumprod().plot()
(RMW['RMW']/100 + 1).cumprod().plot()
(HML['HML']/100 + 1).cumprod().plot()
(kospi['Mkt-RF']/100 + 1).cumprod().plot()
plt.legend()

## CMA 월별 수익률 구하기

#### Size - Inv 포트폴리오
|Size/ Inv|Small|Big|
|---|---|---|
|High Inv|SC|BC|
|2|SN|BN|
|Low Inv|SA|BA|

구하는 식
- CMA = (SC + BC)/2 - (SM + BM)/2  

### 구하는 식의 HML이랑 비슷하므로 바로 코드로 넘어간다.

In [ ]:
cma_list = []

def calculate_return(period):
    print(period)
    big_inv_data = inv.loc[idx[period, big_marcap_ticker],:]['INV']
    small_inv_data = inv.loc[idx[period, small_marcap_ticker],:]['INV']
    
    # 3분위수로 나눈다.
    big_inv_quantile = pd.qcut(big_inv_data, q=3, labels=['BA', 'BN', 'BC']).to_frame()
    small_inv_quantile = pd.qcut(small_inv_data, q=3, labels=['SA', 'SN', 'SC']).to_frame()
    
    bc_ticker = big_inv_quantile[big_inv_quantile['INV'].astype(str) == 'BC'].index.get_level_values('ticker')
    sc_ticker = small_inv_quantile[small_inv_quantile['INV'].astype(str) == 'SC'].index.get_level_values('ticker')
    
    ba_ticker = big_inv_quantile[big_inv_quantile['INV'].astype(str) == 'BA'].index.get_level_values('ticker')
    sa_ticker = small_inv_quantile[small_inv_quantile['INV'].astype(str) == 'SA'].index.get_level_values('ticker')
    
    cma = (get_year_return(period, bc_ticker) + get_year_return(period, sc_ticker))/2 - (get_year_return(period, ba_ticker) + get_year_return(period, sa_ticker))/2
    cma = cma.to_frame()
    cma.columns = ['CMA']
    return cma

for period in period_index:
    cma_list.append(calculate_return(period))

In [ ]:
CMA = pd.concat(cma_list).sort_index()

In [ ]:
(SMB['SMB']/100 + 1).cumprod().plot()
(CMA['CMA']/100 + 1).cumprod().plot()
(RMW['RMW']/100 + 1).cumprod().plot()
(HML['HML']/100 + 1).cumprod().plot()
(kospi['Mkt-RF']/100 + 1).cumprod().plot()
plt.legend()

In [ ]:
CMA

In [ ]:
fama_french_5_factor_korea = pd.concat([kospi, SMB, HML, RMW, CMA], axis=1).dropna()

In [ ]:
fama_french_5_factor_korea

## Fama - French 5요인 저장

In [ ]:
DATA_STORE = Path('assets.h5')

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('fama_french/korea', fama_french_5_factor_korea)

## 미국 Fama - French 5요인 비교

In [ ]:
ff_factor = 'F-F_Research_Data_5_Factors_2x3'
ff_factor_data = pdr.DataReader(ff_factor, 'famafrench', start='1995')[0]

In [ ]:
ff_factor_data

In [ ]:
(ff_factor_data[['SMB', 'HML', 'RMW', 'CMA']]/100 + 1).cumprod().plot()

In [ ]:
ff_factor_data

## 결론
- 기존의 미국 fama french 5요인을 보면 한국 Fama French와는 많이 다른것을 알 수 있다.


## 이유
- 이유는 미국 Fama - French 5요인 데이터는 value weight portfolio로 구성했고 국내 Fama - French는 equal weight portfolio로 구성했으므로 전자보다 시가총액이 작은 주식의 영향력이 커저서 그렇다.